# Data Preparation

## Imports

In [225]:
import pandas as pd
import numpy as np
import ast
import pytz
#import remyenv
import matplotlib.pyplot as plt
import xgboost as xgb 
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")


## Read files and concatenate ID orderbooks
We want to train the model on data from 2022-2023, and then test on the year of 2024

In [ ]:
# This is only data from january 1. 2023. Should be the q4 of 2022, but might not be avialble. Are said something about this in a email
q4_22_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_22q4.csv')

In [226]:
#da_df = pd.read_excel('Data\DA_prices_2023.xlsx')
#da_df.to_parquet('Data/da_23.parquet')
da_df = pd.read_parquet('Data/da_23.parquet')

In [228]:
da_df

,Delivery Start (CET),Delivery End (CET),AT,BE,DK1,DK2,EE,FI,FR,GER,...,NO1,NO2,NO3,NO4,NO5,PL,SE1,SE2,SE3,SE4
0,01.01.2023 00:00:00,01.01.2023 01:00:00,12.06,-4.39,2.01,2.01,2.01,2.01,0.00,-5.17,...,119.32,119.32,23.35,23.35,119.32,18.09,2.01,2.01,2.01,2.01
1,01.01.2023 01:00:00,01.01.2023 02:00:00,-0.10,-1.75,1.38,1.38,1.38,1.38,-0.10,-1.07,...,108.83,108.83,23.28,23.28,108.83,5.75,1.38,1.38,1.38,1.38
2,01.01.2023 02:00:00,01.01.2023 03:00:00,-0.66,-1.46,0.09,0.09,0.09,0.09,-1.33,-1.47,...,102.39,102.39,23.37,23.37,102.39,5.27,0.09,0.09,0.09,0.09
3,01.01.2023 03:00:00,01.01.2023 04:00:00,-1.99,-5.27,0.08,0.08,0.08,0.08,-4.08,-5.08,...,92.36,92.36,23.67,23.67,92.36,5.74,0.08,0.08,0.08,0.08
4,01.01.2023 04:00:00,01.01.2023 05:00:00,-2.42,-4.41,0.05,0.05,0.05,0.05,-4.16,-4.49,...,82.66,82.66,24.12,24.12,82.66,5.22,0.05,0.05,0.05,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,31.12.2023 19:00:00,31.12.2023 20:00:00,19.57,20.26,39.64,39.64,59.99,59.99,17.75,9.00,...,62.88,62.88,39.64,38.14,62.88,46.06,39.64,39.64,39.64,39.64
8756,31.12.2023 20:00:00,31.12.2023 21:00:00,17.20,19.65,34.89,34.89,40.99,40.99,16.43,7.95,...,62.60,62.60,34.89,34.89,62.60,35.01,34.89,34.89,34.89,34.89
8757,31.12.2023 21:00:00,31.12.2023 22:00:00,8.20,9.00,29.60,29.60,53.83,53.83,8.16,6.00,...,62.25,62.25,29.60,29.60,62.25,41.92,29.60,29.60,29.60,29.60
8758,31.12.2023 22:00:00,31.12.2023 23:00:00,10.68,10.68,28.67,28.67,55.49,55.49,10.68,10.68,...,61.77,61.77,28.67,28.67,61.77,48.27,28.67,28.67,28.67,28.67


In [ ]:
## Just to test the functions I want to create a small dataframe to test on, because of time
q1_23_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_23q1.csv')
q1_23_df['dlvryStart'] = pd.to_datetime(q1_23_df['dlvryStart'])

test_df = q1_23_df[q1_23_df['dlvryStart'] < '2023-01-05 01:00:00+00:00']

In [229]:
test_df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask
0,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:00:00.000 +0000,"{'X2790347200','X2790347678','X2790883922','X2...","{'X2791183349','X2791179026','X2790864410','X2...","{81,81,0,0,-20,-24,-90,-90,-91,-93}","{8839,8840,8940,8949,9000,11400,11400,11500,11...","{5000,1000,5000,2000,600,5000,700,5000,7000,5000}","{5000,10000,10000,5000,3000,6000,1000,9000,500..."
1,NX_342255,PH-20230101-04,2023-01-01 02:00:00+00:00,2023-01-01 03:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:27:03.301 +0000,2022-12-31 23:00:00.000 +0000,"{'X2790347441','X2790347679','X2790891201','X2...","{'X2790630460','X2789785962','X2790746637','X2...","{51,51,8,-90,-100,-150,-150,-150,-159,-167}","{8988,9000,10298,10299,10300,11100,11236,11236...","{5000,1000,10000,5000,5000,5000,1600,7000,3000...","{5000,3000,1000,5000,10000,14000,10000,10000,5..."
2,NX_342262,PH-20230101-05,2023-01-01 03:00:00+00:00,2023-01-01 04:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:27:04.578 +0000,2022-12-31 23:00:00.000 +0000,"{'X2790347866','X2789796016','X2790894017','X2...","{'X2790630583','X2789785963','X2790746786','X2...","{-90,-100,-120,-140,-149,-220,-243,-244,-300,-...","{8988,9000,10298,10298,10299,10300,11100,12500...","{5000,5000,100,1500,3000,1100,300,7000,1500,2000}","{5000,3000,1400,10000,5000,10000,14000,5000,16..."
3,NX_342269,PH-20230101-06,2023-01-01 04:00:00+00:00,2023-01-01 05:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:27:06.345 +0000,2022-12-31 23:00:00.000 +0000,"{'X2790348277','X2789796017','X2790895679','X2...","{'X2790776638','X2790690396','X2789544087','X2...","{-90,-100,-290,-290,-390,-391,-455,-459,-474,-...","{10299,10300,11100,14500,14900,15000,15300,155...","{5000,5000,1000,3000,600,7000,1200,19000,2000,...","{14000,10000,14000,10000,33000,10000,20000,500..."
4,NX_342276,PH-20230101-07,2023-01-01 05:00:00+00:00,2023-01-01 06:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:27:08.229 +0000,2022-12-31 23:00:00.000 +0000,"{'X2790894453','X2790860053','X2789796018','X2...","{'X2791171856','X2790865990','X2791254728','X2...","{0,-90,-100,-300,-300,-419,-419,-420,-430,-440}","{7769,7770,8930,10020,10398,10400,10800,10900,...","{100,5000,5000,4500,200,3900,200,1500,1500,1300}","{500,2800,2300,5000,500,5000,15000,4000,15000,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117965,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:55:00.000 +0000,"{'X2811819380','X2811815186','X2811819245','X2...","{'X2811815201','X2811815183','X2811752644','X2...","{6813,6803,6631,6622,6611,6600,6200,5993,5650,...","{6900,6900,7000,7559,8000,8766,8767,8776,8778,...","{1000,1000,5000,3000,3000,14000,1300,200,5000,...","{2000,1000,28500,5000,5000,5000,4000,3000,4000..."
117988,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:56:00.000 +0000,"{'X2811822208','X2811822203','X2811822204','X2...","{'X2811819613','X2811752644','X2811787309','X2...","{6866,6856,6856,6855,6622,6611,6600,6200,5993,...","{7000,7000,7559,8000,9894,9900,9990,9999,10482...","{400,300,1100,1000,3000,3000,14000,1300,200,5000}","{1000,28000,5000,5000,1000,1000,4000,3000,4000..."
118011,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00.000 +0000,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:57:00.000 +0000,"{'X2811823432','X2811823424','X2811823425','X2...","{'X2811823673','X2811822815','X2811752644','X2...","{6877,6867,6867,6866,6675,6622,6611,6600,6200,...","{6913,6950,7000,7559,8000,8000,9868,9894,9894,...","{100,1100,300,1000,200,3000,3000,14000,1300,5000}","{1000,4000,28000,5000,5000,46500,4000,1000,200..."
118034,NX_343220,PH-20230105-02,2023-01-05 00:00:00

In [ ]:
# 2023-01-01 01:00 - 2023-04-01 21:00
q1_23_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_23q1.csv')

# 2023-04-01 00:00 - 2023-07-01 21:00
q2_23_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_23q2.csv')

# 2023-07-01 00:00 - 2023-10-01 21:00
q3_23_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_23q3.csv')

# 2023-10-01 00:00 - 2024-01-01 22:00
q4_23_df = pd.read_csv('Data\intraday_orderbook_snapshots_no3_23q4.csv')

In [ ]:
# Concatenate the ID orderbooks
orderbook_df = pd.concat([q1_23_df, q2_23_df, q3_23_df, q4_23_df])
orderbook_df.to_parquet('Data/orderbook.parquet')

In [ ]:
orderbook_df = pd.read_parquet('Data/orderbook.parquet')

In [ ]:
orderbook_df.dtypes

## Convert columns to datetime

In [ ]:
# The coulumns dlvryStart, dlvryEnd and time are in string format. We need to convert them to datetime format.|
orderbook_df['dlvryStart'] = pd.to_datetime(orderbook_df['dlvryStart'])
orderbook_df['dlvryEnd'] = pd.to_datetime(orderbook_df['dlvryEnd'])
orderbook_df['time'] = pd.to_datetime(orderbook_df['time'])

In [231]:
# The coulumns dlvryStart, dlvryEnd and time are in string format. We need to convert them to datetime format.|
test_df['dlvryStart'] = pd.to_datetime(test_df['dlvryStart'])
test_df['dlvryEnd'] = pd.to_datetime(test_df['dlvryEnd'])
test_df['time'] = pd.to_datetime(test_df['time'])

In [232]:
# Convert string columns to datetime objects (without timezone info initially)
da_df['Delivery Start (CET)'] = pd.to_datetime(da_df['Delivery Start (CET)'], format="%d.%m.%Y %H:%M:%S")
da_df['Delivery End (CET)'] = pd.to_datetime(da_df['Delivery End (CET)'], format="%d.%m.%Y %H:%M:%S")

# Localize to CET timezone
cet = pytz.timezone('CET')
da_df['Delivery Start (CET)'] = da_df['Delivery Start (CET)'].dt.tz_localize(cet, ambiguous='infer')
da_df['Delivery End (CET)'] = da_df['Delivery End (CET)'].dt.tz_localize(cet, ambiguous='infer')

# Convert from CET to UCT
da_df['Delivery Start (UTC)'] = da_df['Delivery Start (CET)'].dt.tz_convert('UTC')
da_df['Delivery End (UTC)'] = da_df['Delivery End (CET)'].dt.tz_convert('UTC')

In [233]:
da_df

,Delivery Start (CET),Delivery End (CET),AT,BE,DK1,DK2,EE,FI,FR,GER,...,NO3,NO4,NO5,PL,SE1,SE2,SE3,SE4,Delivery Start (UTC),Delivery End (UTC)
0,2023-01-01 00:00:00+01:00,2023-01-01 01:00:00+01:00,12.06,-4.39,2.01,2.01,2.01,2.01,0.00,-5.17,...,23.35,23.35,119.32,18.09,2.01,2.01,2.01,2.01,2022-12-31 23:00:00+00:00,2023-01-01 00:00:00+00:00
1,2023-01-01 01:00:00+01:00,2023-01-01 02:00:00+01:00,-0.10,-1.75,1.38,1.38,1.38,1.38,-0.10,-1.07,...,23.28,23.28,108.83,5.75,1.38,1.38,1.38,1.38,2023-01-01 00:00:00+00:00,2023-01-01 01:00:00+00:00
2,2023-01-01 02:00:00+01:00,2023-01-01 03:00:00+01:00,-0.66,-1.46,0.09,0.09,0.09,0.09,-1.33,-1.47,...,23.37,23.37,102.39,5.27,0.09,0.09,0.09,0.09,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00
3,2023-01-01 03:00:00+01:00,2023-01-01 04:00:00+01:00,-1.99,-5.27,0.08,0.08,0.08,0.08,-4.08,-5.08,...,23.67,23.67,92.36,5.74,0.08,0.08,0.08,0.08,2023-01-01 02:00:00+00:00,2023-01-01 03:00:00+00:00
4,2023-01-01 04:00:00+01:00,2023-01-01 05:00:00+01:00,-2.42,-4.41,0.05,0.05,0.05,0.05,-4.16,-4.49,...,24.12,24.12,82.66,5.22,0.05,0.05,0.05,0.05,2023-01-01 03:00:00+00:00,2023-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00+01:00,2023-12-31 20:00:00+01:00,19.57,20.26,39.64,39.64,59.99,59.99,17.75,9.00,...,39.64,38.14,62.88,46.06,39.64,39.64,39.64,39.64,2023-12-31 18:00:00+00:00,2023-12-31 19:00:00+00:00
8756,2023-12-31 20:00:00+01:00,2023-12-31 21:00:00+01:00,17.20,19.65,34.89,34.89,40.99,40.99,16.43,7.95,...,34.89,34.89,62.60,35.01,34.89,34.89,34.89,34.89,2023-12-31 19:00:00+00:00,2023-12-31 20:00:00+00:00
8757,2023-12-31 21:00:00+01:00,2023-12-31 22:00:00+01:00,8.20,9.00,29.60,29.60,53.83,53.83,8.16,6.00,...,29.60,29.60,62.25,41.92,29.60,29.60,29.60,29.60,2023-12-31 20:00:00+00:00,2023-12-31 21:00:00+00:00
8758,2023-12-31 22:00:00+01:00,2023-12-31 23:00:00+01:00,10.68,10.68,28.67,28.67,55.49,55.49,10.68,10.68,...,28.67,28.67,61.77,48.27,28.67,28.67,28.67,28.67,2023-12-31 21:00:00+00:00,2023-12-31 22:00:00+00:00


In [234]:
# Remove the CET columns
da_df = da_df.drop(columns=['Delivery Start (CET)', 'Delivery End (CET)'])

In [235]:
da_df

,AT,BE,DK1,DK2,EE,FI,FR,GER,LT,LV,...,NO3,NO4,NO5,PL,SE1,SE2,SE3,SE4,Delivery Start (UTC),Delivery End (UTC)
0,12.06,-4.39,2.01,2.01,2.01,2.01,0.00,-5.17,2.01,2.01,...,23.35,23.35,119.32,18.09,2.01,2.01,2.01,2.01,2022-12-31 23:00:00+00:00,2023-01-01 00:00:00+00:00
1,-0.10,-1.75,1.38,1.38,1.38,1.38,-0.10,-1.07,1.38,1.38,...,23.28,23.28,108.83,5.75,1.38,1.38,1.38,1.38,2023-01-01 00:00:00+00:00,2023-01-01 01:00:00+00:00
2,-0.66,-1.46,0.09,0.09,0.09,0.09,-1.33,-1.47,0.09,0.09,...,23.37,23.37,102.39,5.27,0.09,0.09,0.09,0.09,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00
3,-1.99,-5.27,0.08,0.08,0.08,0.08,-4.08,-5.08,0.08,0.08,...,23.67,23.67,92.36,5.74,0.08,0.08,0.08,0.08,2023-01-01 02:00:00+00:00,2023-01-01 03:00:00+00:00
4,-2.42,-4.41,0.05,0.05,0.05,0.05,-4.16,-4.49,0.05,0.05,...,24.12,24.12,82.66,5.22,0.05,0.05,0.05,0.05,2023-01-01 03:00:00+00:00,2023-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,19.57,20.26,39.64,39.64,59.99,59.99,17.75,9.00,59.99,59.99,...,39.64,38.14,62.88,46.06,39.64,39.64,39.64,39.64,2023-12-31 18:00:00+00:00,2023-12-31 19:00:00+00:00
8756,17.20,19.65,34.89,34.89,40.99,40.99,16.43,7.95,40.99,40.99,...,34.89,34.89,62.60,35.01,34.89,34.89,34.89,34.89,2023-12-31 19:00:00+00:00,2023-12-31 20:00:00+00:00
8757,8.20,9.00,29.60,29.60,53.83,53.83,8.16,6.00,53.83,53.83,...,29.60,29.60,62.25,41.92,29.60,29.60,29.60,29.60,2023-12-31 20:00:00+00:00,2023-12-31 21:00:00+00:00
8758,10.68,10.68,28.67,28.67,55.49,55.49,10.68,10.68,55.49,55.49,...,28.67,28.67,61.77,48.27,28.67,28.67,28.67,28.67,2023-12-31 21:00:00+00:00,2023-12-31 22:00:00+00:00


## Merge ID orderbook with DA prices

In [ ]:
# Merge the ID orderbook with the DA prices.
df = pd.merge(orderbook_df, da_df, left_on='dlvryStart', right_on='Delivery Start (UTC)', how='left')

In [100]:
# Merge the test orderbook with the DA prices.
df = pd.merge(test_df, da_df, left_on='dlvryStart', right_on='Delivery Start (UTC)', how='left')

In [8]:
df.tail(10)

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,NO3,NO4,NO5,PL,SE1,SE2,SE3,SE4,Delivery Start (UTC),Delivery End (UTC)
104870,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:50:00+00:00,"{'X2811807862','X2811807884','X2811808225','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104871,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:51:00+00:00,"{'X2811810533','X2811810084','X2811798894','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104872,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:52:00+00:00,"{'X2811813089','X2811812447','X2811798894','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104873,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:53:00+00:00,"{'X2811814377','X2811815186','X2811814612','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104874,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:54:00+00:00,"{'X2811817043','X2811815186','X2811814612','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104875,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:55:00+00:00,"{'X2811819380','X2811815186','X2811819245','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104876,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:56:00+00:00,"{'X2811822208','X2811822203','X2811822204','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104877,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:57:00+00:00,"{'X2811823432','X2811823424','X2811823425','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104878,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:58:00+00:00,"{'X2811826255','X2811826246','X2811825821','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00
104879,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:59:00+00:00,"{'X2811827634','X2811827635','X2811827628','X2...",...,28.15,28.15,96.94,76.62,28.15,28.15,28.15,28.15,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00


## Remove rows where DA price is NaN
DA prices go a little bit further than the orderbook. Remove these

In [101]:
df = df.dropna(subset=['NO3'])

In [ ]:
df

## Write merged orderbook to parquet

In [ ]:
df.to_parquet('Data/merged_orderbook.parquet')

## Start here
We can start here each time

In [2]:
df = pd.read_parquet('Data/merged_orderbook.parquet')

In [4]:
df.head(5)

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,NO3,NO4,NO5,PL,SE1,SE2,SE3,SE4,Delivery Start (UTC),Delivery End (UTC)
0,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:00:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,23.37,23.37,102.39,5.27,0.09,0.09,0.09,0.09,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00
1,NX_342255,PH-20230101-04,2023-01-01 02:00:00+00:00,2023-01-01 03:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:03.301 +0000,2022-12-31 23:00:00+00:00,"{'X2790347441','X2790347679','X2790891201','X2...",...,23.67,23.67,92.36,5.74,0.08,0.08,0.08,0.08,2023-01-01 02:00:00+00:00,2023-01-01 03:00:00+00:00
2,NX_342262,PH-20230101-05,2023-01-01 03:00:00+00:00,2023-01-01 04:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:04.578 +0000,2022-12-31 23:00:00+00:00,"{'X2790347866','X2789796016','X2790894017','X2...",...,24.12,24.12,82.66,5.22,0.05,0.05,0.05,0.05,2023-01-01 03:00:00+00:00,2023-01-01 04:00:00+00:00
3,NX_342269,PH-20230101-06,2023-01-01 04:00:00+00:00,2023-01-01 05:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:06.345 +0000,2022-12-31 23:00:00+00:00,"{'X2790348277','X2789796017','X2790895679','X2...",...,32.24,24.67,80.58,6.14,0.08,0.08,0.08,0.08,2023-01-01 04:00:00+00:00,2023-01-01 05:00:00+00:00
4,NX_342276,PH-20230101-07,2023-01-01 05:00:00+00:00,2023-01-01 06:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:08.229 +0000,2022-12-31 23:00:00+00:00,"{'X2790894453','X2790860053','X2789796018','X2...",...,35.93,24.54,77.65,5.31,0.09,0.09,0.09,0.09,2023-01-01 05:00:00+00:00,2023-01-01 06:00:00+00:00


In [ ]:
df.dtypes

## Formatting the DF

In [106]:
# Set the delivery hour to be the index. This will probably not be needed
def setIndex(df):
    df = df.set_index(['dlvryStart', 'time'])
    return df

# Sort by dlvryStart and time. Meaning that we get dlvryStart at e.g. 23 with time=14.01, 14.02 etc before dlvryStart at 00 with time=14.01, 14.02 etc
def sortByDlvryStart(df):
    # Sort by time within each index group
    df = df.sort_values(by=['dlvryStart', 'time'])
    df = df.groupby(df['dlvryStart'], group_keys=False).apply(lambda x: x.sort_values(by='time'))
    return df

# Drop unnecessary columns
def dropColumns(df):
    df = df.drop(columns=['contractId', 'contractName', 'dlvryEnd', 'dlvryAreaId', 'marketId', 'area', 'recorded', 'Delivery Start (UTC)', 'Delivery End (UTC)'])
    return df
'''
# Used in the apply_process_string function
def process_string(input_str):
    # Check if the string is empty or contains only curly braces
    if input_str in ['{}', '']:
        return []   
    else:
        # Remove the curly braces, split by commas, and convert to float
        return [float(x) for x in input_str[1:-1].split(',')]
'''

# Used in the apply_process_string function
def process_string(input_str):
    # Check if the string is empty or contains only curly braces
    if input_str in ['{}', '']:
        return []   
    else:
        # Direct string slicing and list comprehension in one pass
        content = input_str[1:-1]
        if not content:
            return []
    
        # Split and convert in one operation
        return [float(x) for x in content.split(',')]

# Converts the string representation of a list to a list of floats
def apply_process_string(df):
    cols = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask']
    for col in cols:
        # Apply the process_string function to each element in the column
        df[col] = df[col].apply(lambda x: process_string(x) if isinstance(x, str) else x)
    return df

def retrieveBestOffers(df):
    columns = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask']
    for col in columns:
        df[col + '_best'] = df[col].apply(lambda x: x[0] if len(x) > 0 else pd.NA)
    # Check if there are any bids/asks
    df['exists_bid'] = df['orderPrice_bid'].apply(lambda x: 1 if len(x) > 0 else 0)
    df['exists_ask'] = df['orderPrice_ask'].apply(lambda x: 1 if len(x) > 0 else 0)
    return df

def fixPriceVolumeMagnitude(df):
    # Define the columns to be processed
    #price_cols = ['orderPrice_bid', 'orderPrice_ask', 'orderPrice_bid_best', 'orderPrice_ask_best']
    #qty_cols = ['orderQty_bid', 'orderQty_ask', 'orderQty_bid_best', 'orderQty_ask_best']
    price_cols = ['orderPrice_bid_best', 'orderPrice_ask_best']
    qty_cols = ['orderQty_bid_best', 'orderQty_ask_best']
    
    for col in price_cols:
        df[col] = df[col].apply(lambda x: x / 100 if pd.notna(x) else x)

    for col in qty_cols:
        df[col] = df[col].apply(lambda x: x / 1000 if pd.notna(x) else x)

    return df

# Rename column names for better readability
def changeName(df):
    df = df.rename(columns={'NO3 Price (EUR)': 'DA_price'})
    return df

# Split delivery hour and time to month, day, hour etc
def createTimeColumns(df):
    df['dlvryMonth'] = df['dlvryStart'].dt.month
    df['dlvryDay'] = df['dlvryStart'].dt.day
    df['dlvryHour'] = df['dlvryStart'].dt.hour
    df['dlvryWeekday'] = df['dlvryStart'].dt.weekday

    df['timeMonth'] = df['time'].dt.month
    df['timeDay'] = df['time'].dt.day
    df['timeHour'] = df['time'].dt.hour
    df['timeWeekday'] = df['time'].dt.weekday
    df['timeMinute'] = df['time'].dt.minute
    return df

# For each row it find the best price that appears later in the LOB for the same delivery hour.
def findFutureBestOffers(df):
    # Replace NaNs with -inf for bids and +inf for asks, because there is some issues with compariosn of NaNs to ordinary values
    df['orderPrice_bid_best'] = df['orderPrice_bid_best'].fillna(-np.inf)
    df['orderPrice_ask_best'] = df['orderPrice_ask_best'].fillna(np.inf)

    # Initalize new columns
    new_cols = ['better_bid_later', 'better_ask_later', 'future_best_bid', 'future_best_ask']

    delivery_hours = df.groupby('dlvryStart')
    for _, group in delivery_hours:
        reverse_group = group.iloc[::-1].copy() # Reverses the order
        reverse_group['cum_max_bid'] = reverse_group['orderPrice_bid_best'].cummax() # Finds the best price seen so far
        reverse_group['cum_min_ask'] = reverse_group['orderPrice_ask_best'].cummin() # Finds the best price seen so far

        # Restore the original order
        forward_group = reverse_group.iloc[::-1].copy()

        # Set future best prices
        forward_group['future_best_bid'] = forward_group['cum_max_bid'].shift(-1)
        forward_group['future_best_ask'] = forward_group['cum_min_ask'].shift(-1)
        
        # Compare current prices with future best prices
        forward_group['better_bid_later'] = (
            forward_group['cum_max_bid'].shift(-1) > forward_group['orderPrice_bid_best']
        ).astype(int)

        forward_group['better_ask_later'] = (
            forward_group['cum_min_ask'].shift(-1) < forward_group['orderPrice_ask_best']
        ).astype(int)

        # Convert infinities back to NaN for clean output
        forward_group['future_best_bid'] = forward_group['future_best_bid'].replace(-np.inf, pd.NA)
        forward_group['future_best_ask'] = forward_group['future_best_ask'].replace(np.inf, pd.NA)
        
        # Update original dataframe
        df.loc[forward_group.index, new_cols] = forward_group[new_cols]
        
    # Restore original NaN values in source columns
    df['orderPrice_bid_best'] = df['orderPrice_bid_best'].replace(-np.inf, np.nan)
    df['orderPrice_ask_best'] = df['orderPrice_ask_best'].replace(np.inf, np.nan)
    
    return df

# For each delivery hour. I want to find the best far price, aka the best price from market opening until six hours before delivery. 
def findBestFarPrice(df):
    df['best_far_bid'] = pd.NA
    df['best_far_ask'] = pd.NA
    groups = df.groupby('dlvryStart')
    for _, group in groups:
        # Find the cutoff timestamp (6 hours before delivery)
        cutoff_time = group['dlvryStart'].iloc[0] - pd.Timedelta(hours=6)

        # Filter data up to cutoff time
        far_market_data = group[group['time'] <= cutoff_time]

        # Calculate the best far prices
        best_far_bid = far_market_data['orderPrice_bid_best'].max(skipna=True)
        best_far_ask = far_market_data['orderPrice_ask_best'].min(skipna=True)

        # Assign values to the original DataFrame
        df.loc[group.index, 'best_far_bid'] = best_far_bid
        df.loc[group.index, 'best_far_ask'] = best_far_ask

    return df

# Create a column that shows time until delivery in minutes. Might capture something
def createTimeUntilDeliveryCol(df):
    df['minutesFromDelivery'] = (df['dlvryStart'] - df['time']).dt.total_seconds() / 60 # Both are datetime objects
    return df

# Drop the last minute of trading. We cant take trades after the last minute of trading, so it makes sense to exclude this row. 
def dropLastMinuteOfTrading(df):
    return df.groupby('dlvryStart', group_keys=False).apply(lambda group: group.iloc[:-1])

# Create intervals. Only keep rows every 30 minutes when more than 6 hours from delivery. Use 15-minute intervals when 2-6 hours from delivery. Use 5-minute intervals when less than 2 hours from delivery
def createIntervals(df):
    df['hoursUntilDelivery'] = df['minutesFromDelivery'] / 60
    
     # Filter rows based on time_to_delivery using boolean indexing
    filtered_df = df[
        ((df['hoursUntilDelivery'] > 6) & (df['time'].dt.minute % 30 == 0)) |  # Every 30 min when > 6h
        ((df['hoursUntilDelivery'] >= 2) & (df['hoursUntilDelivery'] <= 6) & (df['time'].dt.minute % 15 == 0)) |  # Every 15 min when 2-6h
        ((df['hoursUntilDelivery'] < 2) & (df['time'].dt.minute % 5 == 0))  # Every 5 min when < 2h
    ]
    
    return filtered_df

def create24hLag(df):
    # Create columns for 24h ago
    df['dlvryStart_24h_ago'] = df['dlvryStart'] - pd.Timedelta(days=1)
    df['time_24h_ago'] = df['time'] - pd.Timedelta(days=1)
    
    # Merge to get values from 24h ago
    df_with_lags = pd.merge(
        df,
        df[['dlvryStart', 'time', 'future_best_bid', 'future_best_ask']],
        left_on=['dlvryStart_24h_ago', 'time_24h_ago'],
        right_on=['dlvryStart', 'time'],
        how='left',
        suffixes=('', '_24h_ago')
    )
    
    # Drop temporary columns
    df_with_lags = df_with_lags.drop(['dlvryStart_24h_ago', 'time_24h_ago', 'dlvryStart_24h_ago'], axis=1)
    
    return df_with_lags




 

In [ ]:
#df = setIndex(df) # Maybe not needed
#df = dropColumns(df) # Maybe better to just exclude in the FEATURES

In [11]:
df[['orderPrice_bid', 'orderPrice_bid_best']].head()

,orderPrice_bid,orderPrice_bid_best
0,"{81,81,0,0,-20,-24,-90,-90,-91,-93}",{
22,"{81,81,0,0,-20,-24,-90,-90,-91,-93}",{
44,"{81,81,0,0,-20,-24,-90,-90,-91,-93}",{
66,"{81,81,0,0,-20,-24,-90,-90,-91,-93}",{
88,"{81,81,0,0,-20,-24,-90,-90,-91,-93}",{


In [105]:
df[df['dlvryStart'] == '2023-01-04 02:00:00+00:00'][['dlvryStart', 'time', 'orderPrice_bid', 'orderPrice_bid_best', 'orderPrice_ask', 'orderPrice_ask_best', 'best_far_bid', 'future_best_bid']]

KeyError: "['orderPrice_bid_best', 'orderPrice_ask_best', 'best_far_bid', 'future_best_bid'] not in index"

In [36]:
df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,orderQty_ask_best,exists_bid,exists_ask,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask,minutesFromDelivery
0,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:00:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,120.0
22,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:01:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,119.0
44,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:02:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,118.0
66,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:03:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,117.0
88,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:04:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104875,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:55:00+00:00,"{'X2811819380','X2811815186','X2811819245','X2...",...,2.0,1,1,30.12,28.15,1.0,0.0,69.73,69.13,65.0
104876,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:56:00+00:00,"{'X2811822208','X2811822203','X2811822204','X2...",...,1.0,1,1,30.12,28.15,1.0,1.0,69.73,69.13,64.0
104877,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:57:00+00:00,"{'X2811823432','X2811823424','X2811823425','X2...",...,1.0,1,1,30.12,28.15,1.0,0.0,69.73,70.0,63.0
104878,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:58:00+00:00,"{'X2811826255','X2811826246','X2811825821','X2...",...,28.0,1,1,30.12,28.15,1.0,0.0,69.73,75.59,62.0


In [108]:
df = createTimeColumns(df) # Creates columns out of dlvryStart and time
df = sortByDlvryStart(df) # Sorts the dataframe by dlvyStart and then time
df = apply_process_string(df) # Converts orderPrice_bid etc from '{}' to list []. Easier to extract the elements then
df = retrieveBestOffers(df) # Gets the best prices and the corresponding quantities and stores them in new columns

#df.to_parquet('Data/formatted_orderbook.parquet')
#df = pd.read_parquet('Data/formatted_orderbook.parquet')

df = fixPriceVolumeMagnitude(df) # Divde prices by 100 and quantities by 1000 to get euro/MWh and MWh

#df = changeName(df) # Changes the name of columns for better readibility
df = findBestFarPrice(df) # Finds the best price from market opening until six hours before delivery
df = findFutureBestOffers(df) # Finds the best best price that appears later in the orderbook for the same delivery hour
df = createTimeUntilDeliveryCol(df)

df = dropLastMinuteOfTrading(df)
#df = createIntervals(df)
df = create24hLag(df)

#df.to_parquet('Data/formatted_orderbook.parquet')
#df = pd.read_parquet('Data/formatted_orderbook.parquet')





In [109]:
df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,exists_ask,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask,minutesFromDelivery,future_best_bid_24h_ago,future_best_ask_24h_ago
0,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:00:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,1,NaN,NaN,0.0,1.0,0.81,88.34,120.0,NaN,NaN
1,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:01:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,1,NaN,NaN,0.0,1.0,0.81,88.34,119.0,NaN,NaN
2,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:02:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,1,NaN,NaN,0.0,1.0,0.81,88.34,118.0,NaN,NaN
3,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:03:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,1,NaN,NaN,0.0,1.0,0.81,88.34,117.0,NaN,NaN
4,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:04:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,1,NaN,NaN,0.0,1.0,0.81,88.34,116.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104779,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:54:00+00:00,"{'X2811817043','X2811815186','X2811814612','X2...",...,1,30.12,28.15,1.0,0.0,69.73,69.0,66.0,79.3,83.0
104780,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:55:00+00:00,"{'X2811819380','X2811815186','X2811819245','X2...",...,1,30.12,28.15,1.0,0.0,69.73,69.13,65.0,79.3,83.0
104781,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:56:00+00:00,"{'X2811822208','X2811822203','X2811822204','X2...",...,1,30.12,28.15,1.0,1.0,69.73,69.13,64.0,79.3,83.0
104782,NX_343220,PH-20230105-02,2023-01-05 00:00:00+00:00,2023-01-05 01:00:00+00:00,10,N_2,NO3,2024-03-06 10:30:35.371 +0000,2023-01-04 22:57:00+00:00,"{'X2811823432','X2811823424','X2811823425','X2...",...,1,30.12,28.15,1.0,0.0,69.73,70.0,63.0,79.3,83.0


## Handle wind forecasting

In [188]:
# Incorporate external variables into the dataframe
wind_forecasts_df = pd.read_parquet('Data/finishedForecasts.parquet')

In [143]:
wind_forecasts_df.head()

,Forecast Time,valid_time,lead_time_hours,forecast_run,Interpolated,wind_farm,latitude,longitude,wind_speed_10m
0,2022-09-01,2022-09-01 01:00:00,1.0,T00Z,No,Storheia,63.582123,10.176011,0.817069
1,2022-09-01,2022-09-01 19:00:00,19.0,T00Z,No,Hitra,63.232677,8.831443,0.670384
2,2022-09-01,2022-09-01 18:00:00,18.0,T00Z,No,Hitra,63.232677,8.831443,1.122469
3,2022-09-01,2022-09-01 17:00:00,17.0,T00Z,No,Hitra,63.232677,8.831443,2.231902
4,2022-09-01,2022-09-01 16:00:00,16.0,T00Z,No,Hitra,63.232677,8.831443,3.247164


In [42]:
df.head(5)

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,orderQty_ask_best,exists_bid,exists_ask,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask,minutesFromDelivery
0,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:00:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,120.0
22,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:01:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,119.0
44,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:02:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,118.0
66,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:03:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,117.0
88,NX_342247,PH-20230101-03,2023-01-01 01:00:00+00:00,2023-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:27:02.350 +0000,2022-12-31 23:04:00+00:00,"{'X2790347200','X2790347678','X2790883922','X2...",...,5.0,1,1,NaN,NaN,0.0,1.0,0.81,88.34,116.0


In [189]:
def makeUTCsensitive(df):
    df['Forecast Time'] = pd.to_datetime(df['Forecast Time'], utc=True)
    df['valid_time'] = pd.to_datetime(df['valid_time'], utc=True)
    return df

# Sort the wind forecasts by forecast time, then valid time (deliveryHour)
def sortWindForecasts(df):
    # Sort by time within each index group
    df = df.sort_values(by=['Forecast Time', 'valid_time'])
    df = df.groupby(df['Forecast Time'], group_keys=False).apply(lambda x: x.sort_values(by='valid_time'))
    return df

# Now we have one row for each location. I think it would make more sense to have everything in one row
def createOneRowPerForecast(df):
    # Create wide format with one column per wind farm
    df = df.pivot_table(
    index=['Forecast Time', 'valid_time', 'forecast_run', 'lead_time_hours'],
    columns='wind_farm',
    values='wind_speed_10m'
).reset_index()
    
    # Add '_DA_wind' to each wind farm column name
    new_columns = []
    for col in df.columns:
        if col not in ['Forecast Time', 'valid_time', 'forecast_run', 'lead_time_hours']:
            new_columns.append(f"{col}_forecast_wind")
        else:
            new_columns.append(col)
    df.columns = new_columns
    return df

# Only keep the forecasts that were made at 12 for only the hours the following day
def onlyKeepDaForecasts(df):
    df_copy = df.copy()
    # Keep only forecasts made at hour 12
    df_copy = df_copy[df_copy['Forecast Time'].dt.hour == 12]
    #Filter for valid_time being on the next day. Extract date components for comparison
    df_copy['forecast_date'] = df_copy['Forecast Time'].dt.date
    df_copy['valid_date'] = df['valid_time'].dt.date

    # Calculate the next day after each forecast date
    df_copy['next_day'] = pd.to_datetime(df_copy['forecast_date']) + pd.Timedelta(days=1)
    df_copy['next_day'] = df_copy['next_day'].dt.date
    
    # Keep only rows where valid_date is the day after forecast_date
    result_df = df_copy[df_copy['valid_date'] == df_copy['next_day']]

    # Remove the temporary columns we created
    result_df = result_df.drop(columns=['forecast_date', 'valid_date', 'next_day'])

    # Rename columns to append "_DA" to wind farm forecast columns
    new_columns = {}
    for col in result_df.columns:
        if "_forecast_wind" in col:
            new_columns[col] = col.replace("_forecast_wind", "_forecast_wind_DA")

    # Aplly the renaming
    result_df = result_df.rename(columns=new_columns)
    
    return result_df

def findLatestForecast(df_forecast, df_orderbook):
    # First join forecasts with orderbook based on delivery hour
    merged = pd.merge(
        df_orderbook,
        df_forecast,
        left_on='dlvryStart',
        right_on='valid_time', 
        how='left'
    )

    # Filter for forecatss available at order time
    merged = merged[merged['Forecast Time'] <= merged['time']]

    merged = merged.sort_values(['dlvryStart', 'Forecast Time'], ascending=[True, False])

    # Keep only the first (latest) forecast for each order
    final_result = merged.drop_duplicates(subset=['dlvryStart', 'time'], keep='first')

    return final_result

# Calculates the difference between the DA forecast and the latest available forecast 
def calculate_wind_forecast_errors(df):
    # Get all columns ending with "_forecast_wind_DA"
    da_forecast_columns = [col for col in df.columns if col.endswith('_forecast_wind_DA')]
    
    # For each DA forecast column, create the corresponding error column
    for da_col in da_forecast_columns:
        # Get the wind park name from the column name
        wind_park = da_col.replace('_forecast_wind_DA', '')
        
        # Find the latest forecast column (without "_DA")
        latest_col = f"{wind_park}_forecast_wind"
        
        # Only proceed if both columns exist
        if latest_col in df.columns:
            # Calculate the error (DA forecast - latest forecast)
            error_col_name = f"{wind_park}_forecast_wind_error"
            df[error_col_name] = df[da_col] - df[latest_col]
    
    return df


In [ ]:
# This is the general formatting of the wind_forecasts_df
wind_forecasts_df = makeUTCsensitive(wind_forecasts_df)
wind_forecasts_df = sortWindForecasts(wind_forecasts_df)
wind_forecasts_df = createOneRowPerForecast(wind_forecasts_df)

# Create a df that only contains the DA forecasts. This can then be used when we merge with the orderbook
only_da_forecasts = wind_forecasts_df.copy()
only_da_forecasts = onlyKeepDaForecasts(only_da_forecasts)

# Merge the orderbook with the df that only contains DA forecasts. We can then get the DA forecast as an input to the model
merged_df = pd.merge(left=df, right=only_da_forecasts, left_on='dlvryStart', right_on='valid_time')

# Drop the columns that would cause trouble when we try to do a later join
merged_df = merged_df.drop(['Forecast Time', 'valid_time'], axis=1)

latest_forecast_df = wind_forecasts_df.copy()
merged_df = findLatestForecast(latest_forecast_df, merged_df)

# merged_df should now contain the DA forecast, the latest available forecast and the forecast error
merged_df = calculate_wind_forecast_errors(merged_df)



In [175]:
merged_df.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'AT', 'BE', 'DK1', 'DK2', 'EE', 'FI', 'FR', 'GER', 'LT', 'LV', 'NL',
       'NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'PL', 'SE1', 'SE2', 'SE3', 'SE4',
       'Delivery Start (UTC)', 'Delivery End (UTC)', 'dlvryMonth', 'dlvryDay',
       'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour',
       'timeWeekday', 'timeMinute', 'orderPrice_bid_best', 'orderQty_bid_best',
       'orderPrice_ask_best', 'orderQty_ask_best', 'exists_bid', 'exists_ask',
       'best_far_bid', 'best_far_ask', 'better_bid_later', 'better_ask_later',
       'future_best_bid', 'future_best_ask', 'minutesFromDelivery',
       'future_best_bid_24h_ago', 'future_best_ask_24h_ago', 'Forecast Time',
       'valid_time', 'forecast_run', 'lead_time_hours',
       'Geitfjellet_forecas

In [192]:
merged_df.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'AT', 'BE', 'DK1', 'DK2', 'EE', 'FI', 'FR', 'GER', 'LT', 'LV', 'NL',
       'NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'PL', 'SE1', 'SE2', 'SE3', 'SE4',
       'Delivery Start (UTC)', 'Delivery End (UTC)', 'dlvryMonth', 'dlvryDay',
       'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour',
       'timeWeekday', 'timeMinute', 'orderPrice_bid_best', 'orderQty_bid_best',
       'orderPrice_ask_best', 'orderQty_ask_best', 'exists_bid', 'exists_ask',
       'best_far_bid', 'best_far_ask', 'better_bid_later', 'better_ask_later',
       'future_best_bid', 'future_best_ask', 'minutesFromDelivery',
       'future_best_bid_24h_ago', 'future_best_ask_24h_ago', 'forecast_run_x',
       'lead_time_hours_x', 'Geitfjellet_forecast_wind_DA',
       'Hitra_forecast

In [199]:
merged_df[['dlvryStart', 'time', 'Geitfjellet_forecast_wind', 'Geitfjellet_forecast_wind_DA', 'Geitfjellet_forecast_wind_error']].tail(10)

,dlvryStart,time,Geitfjellet_forecast_wind,Geitfjellet_forecast_wind_DA,Geitfjellet_forecast_wind_error
2665596,2023-01-05 00:00:00+00:00,2023-01-04 13:50:00+00:00,14.305414,14.305414,0.0
2665608,2023-01-05 00:00:00+00:00,2023-01-04 13:51:00+00:00,14.305414,14.305414,0.0
2665620,2023-01-05 00:00:00+00:00,2023-01-04 13:52:00+00:00,14.305414,14.305414,0.0
2665632,2023-01-05 00:00:00+00:00,2023-01-04 13:53:00+00:00,14.305414,14.305414,0.0
2665644,2023-01-05 00:00:00+00:00,2023-01-04 13:54:00+00:00,14.305414,14.305414,0.0
2665656,2023-01-05 00:00:00+00:00,2023-01-04 13:55:00+00:00,14.305414,14.305414,0.0
2665668,2023-01-05 00:00:00+00:00,2023-01-04 13:56:00+00:00,14.305414,14.305414,0.0
2665680,2023-01-05 00:00:00+00:00,2023-01-04 13:57:00+00:00,14.305414,14.305414,0.0
2665692,2023-01-05 00:00:00+00:00,2023-01-04 13:58:00+00:00,14.305414,14.305414,0.0
2665704,2023-01-05 00:00:00+00:00,2023-01-04 13:59:00+00:00,14.305414,14.305414,0.0


## Handle transfer capacity file

In [212]:
transfer_cap_df = pd.read_parquet('Data/availableTransferCapacity.parquet')

In [213]:
transfer_cap_df = transfer_cap_df.reset_index()

In [220]:
transfer_cap_df

,Delivery Start (CET),NO3-NO1,NO1-NO3,NO3-NO4,NO4-NO3,NO3-NO5,NO5-NO3,NO3-SE2,SE2-NO3,Delivery Start (UTC)
0,2022-01-01 00:00:00,113,500,400,96,370.0,500.0,600,953,2021-12-31 23:00:00+00:00
1,2022-01-01 01:00:00,125,500,400,93,449.0,500.0,600,941,2022-01-01 00:00:00+00:00
2,2022-01-01 02:00:00,112,500,400,55,464.0,500.0,600,972,2022-01-01 01:00:00+00:00
3,2022-01-01 03:00:00,134,500,400,28,404.0,500.0,600,930,2022-01-01 02:00:00+00:00
4,2022-01-01 04:00:00,172,500,400,36,459.0,500.0,600,845,2022-01-01 03:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...
26299,2024-12-31 19:00:00,91,500,400,38,626.0,500.0,600,872,2024-12-31 18:00:00+00:00
26300,2024-12-31 20:00:00,70,500,400,0,445.0,500.0,600,762,2024-12-31 19:00:00+00:00
26301,2024-12-31 21:00:00,75,500,400,0,213.0,500.0,600,569,2024-12-31 20:00:00+00:00
26302,2024-12-31 22:00:00,112,500,400,21,215.0,500.0,600,471,2024-12-31 21:00:00+00:00


In [219]:
transfer_cap_df['Delivery Start (CET)'] = pd.to_datetime(transfer_cap_df['Delivery Start (CET)'])
transfer_cap_df['Delivery Start (UTC)'] = transfer_cap_df['Delivery Start (CET)'].dt.tz_localize('Europe/Oslo', ambiguous='infer').dt.tz_convert('UTC')


In [221]:
# Merge the df with the transfer cap
merged_df = pd.merge(left=merged_df, right=transfer_cap_df, left_on='dlvryStart', right_on='Delivery Start (UTC)')

In [223]:
merged_df.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'AT', 'BE', 'DK1', 'DK2', 'EE', 'FI', 'FR', 'GER', 'LT', 'LV', 'NL',
       'NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'PL', 'SE1', 'SE2', 'SE3', 'SE4',
       'Delivery Start (UTC)_x', 'Delivery End (UTC)', 'dlvryMonth',
       'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay',
       'timeHour', 'timeWeekday', 'timeMinute', 'orderPrice_bid_best',
       'orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best',
       'exists_bid', 'exists_ask', 'best_far_bid', 'best_far_ask',
       'better_bid_later', 'better_ask_later', 'future_best_bid',
       'future_best_ask', 'minutesFromDelivery', 'future_best_bid_24h_ago',
       'future_best_ask_24h_ago', 'forecast_run_x', 'lead_time_hours_x',
       'Geitfjellet_forecast_wind_DA', 'Hitra_foreca

## Train/Test split

In [ ]:
# Just create a copy of the dataframe
df_copy = df.copy()
train, test = train_test_split(df_copy, test_size=0.2, shuffle=False)

In [ ]:
train

In [ ]:
def filter_random_test_set(test):
    # Ensure 'dlvryStart' is in datetime format
    test['dlvryStart'] = pd.to_datetime(test['dlvryStart'])
    # Extract delivery hour from the 'dlvryStart' column
    test['dlvryHour'] = test['dlvryStart'].dt.hour
    # Group by delivery hour and sample one random row per group
    filtered_test = test.groupby('dlvryHour', group_keys=False).apply(lambda x: x.sample(1))
    # Drop the helper column 'dlvryHour' after filtering
    filtered_test = filtered_test.drop(columns=['dlvryHour'])
    return filtered_test

# Apply the filtering function to the test set
test_filtered = filter_random_test_set(test)
print(test)

In [ ]:
df_copy.columns

In [ ]:
# I think it is better to train a separate model for bid and ask
# I dont think xgboost handles lists as input, so we need to convert the lists to separate columns or just drop them
FEATURES_BID = ['orderPrice_bid_best','orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best', 'best_far_bid', 'best_far_ask', 'DA_price', 'dlvryMonth', 'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour', 'timeWeekday', 'timeMinute', 'minutesFromDelivery', 'exists_bid', 'exists_ask', 'future_best_bid_24h_ago', 'future_best_ask_24h_ago']
FEATURES_ASK = ['orderPrice_bid_best','orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best', 'best_far_bid', 'best_far_ask', 'DA_price', 'dlvryMonth', 'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour', 'timeWeekday', 'timeMinute', 'minutesFromDelivery', 'exists_bid', 'exists_ask', 'future_best_bid_24h_ago', 'future_best_ask_24h_ago']
TARGET_BID = ['future_best_bid']
TARGET_ASK = ['future_best_ask']

In [ ]:
x_train_bid = train[FEATURES_BID]
y_train_bid = train[TARGET_BID]
x_test_bid = test[FEATURES_BID]
y_test_bid = test[TARGET_BID]

x_train_ask = train[FEATURES_ASK]
y_train_ask = train[TARGET_ASK]
x_test_ask = test[FEATURES_ASK]
y_test_ask = test[TARGET_ASK]

## Handling Nan and Inf
If there are no future orders, they get assigned a NaN value. This is a problem for Xgboost, meaning we must convert these NaN values to something. We try to replace NaN with a special value that indicates 'no future orders' and treat it as a valid target value. Edit: almost all of these rows where the future best order is NaN is the last minute of trading, so it makes sense. Should find out when there are no more better trades. Could be useful in the data analysis as well.

In [ ]:
na_rows = df[df['future_best_bid'].isna()]

In [ ]:
na_rows.iloc[15:30]

In [ ]:
print(len(na_rows))

In [ ]:
y_train_bid = y_train_bid.fillna(-999)
y_test_bid = y_test_bid.fillna(-999)

y_train_ask = y_train_ask.fillna(999)
y_test_ask = y_test_ask.fillna(999)

In [ ]:
# Check and clean for NaN in y_train and y_test
# Bids
print("NaN values in y_train_bid before cleaning:", y_train_bid.isna().sum().sum())
print("NaN values in y_test_bid before cleaning:", y_test_bid.isna().sum().sum())

# Asks
print("NaN values in y_train_ask before cleaning:", y_train_ask.isna().sum().sum())
print("NaN values in y_test_ask before cleaning:", y_test_ask.isna().sum().sum())
'''
print("NaN values in x_train before cleaning:", x_train_bid_imputed.isna().sum().sum())
print("NaN values in x_test before cleaning:", x_test_bid.isna().sum().sum())
'''

In [ ]:
# Ensure there are no inf values left in features
print("Infinite values in x_train:", np.isinf(x_train_bid).sum().sum())
print("Infinite values in x_test:", np.isinf(x_test_bid).sum().sum())

## Plot the data
Need to plot the data. Maybe use something like sns.pairplot like in the tennis video. Variables plotted against each other

## Model bid

In [ ]:
# Create model instance
model_bid = XGBRegressor(n_estimators=1000, early_stopping_rounds=50, learning_rate=0.01)
# Fit model
model_bid.fit(x_train_bid, y_train_bid, eval_set=[(x_train_bid, y_train_bid), (x_test_bid, y_test_bid)], verbose=True)
# Make predictions  
preds_bid = model_bid.predict(x_test_bid)

In [ ]:
results_bid = x_test_bid.copy()
results_bid['preds'] = preds_bid
results_bid['actual'] = y_test_bid

In [ ]:
results_bid.head(10)

## Precision


In [ ]:
# Calculate metrics
mse = mean_squared_error(y_test_bid, preds_bid)
mae = mean_absolute_error(y_test_bid, preds_bid)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")


## Feature Importance

In [ ]:
# Try to find out which features are most important
fi = pd.DataFrame(data=model_bid.feature_importances_, index=model_bid.feature_names_in_, columns=['importance'])

In [ ]:
fi.sort_values(by='importance').plot(kind='barh', title='Feature Importance')
plt.show()

## Model ask

In [ ]:
# Create model instance
model_ask = XGBRegressor(n_estimators=1000, early_stopping_rounds=50, max_depth=3, learning_rate=0.1)
# Fit model
model_ask.fit(x_train_ask, y_train_ask_imputed, eval_set=[(x_train_ask, y_train_ask_imputed), (x_test_ask, y_test_ask_imputed)], verbose=True)
# Make predictions  
preds_ask = model_bid.predict(x_test_ask)

In [ ]:
results_ask = x_test_ask.copy()
results_ask['preds'] = preds_ask
results_ask['actual'] = y_test_ask_imputed

In [ ]:
results_ask

## Precision

In [ ]:
# Calculate metrics
mse = mean_squared_error(y_test_bid_imputed, preds_bid)
mae = mean_absolute_error(y_test_bid_imputed, preds_bid)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")

## Feature Importance

In [ ]:
# Try to find out which features are most important
fi = pd.DataFrame(data=model_bid.feature_importances_, index=model_bid.feature_names_in_, columns=['importance'])

In [ ]:
fi.sort_values(by='importance').plot(kind='barh', title='Feature Importance')
plt.show()